## Notebook loads ATP results from tennis-data.co.uk, cleans and saves as single CSV file.
On my laptop with decent internet, this takes about 1 minute to run

In [1]:
import pandas as pd
import numpy as np
import zipfile
import requests
from io import BytesIO
import os
pd.set_option('display.max_columns', None)

In [2]:
# define set of files to download
# @TODO: save each raw file individually so that in future we can
# only download the files we don't already have and the current year
links = ['http://tennis-data.co.uk/2000/2000.xls', 'http://tennis-data.co.uk/2001/2001.xls', 'http://tennis-data.co.uk/2002/2002.xls', 'http://tennis-data.co.uk/2003/2003.xls',
        'http://tennis-data.co.uk/2004/2004.xls', 'http://tennis-data.co.uk/2005/2005.xls', 'http://tennis-data.co.uk/2006/2006.xls', 'http://tennis-data.co.uk/2007/2007.xls',
        'http://tennis-data.co.uk/2008/2008.zip', 'http://tennis-data.co.uk/2009/2009.xls', 'http://tennis-data.co.uk/2010/2010.xls', 'http://tennis-data.co.uk/2011/2011.xls',
        'http://tennis-data.co.uk/2012/2012.xls', 'http://tennis-data.co.uk/2013/2013.xlsx', 'http://tennis-data.co.uk/2014/2014.xlsx', 'http://tennis-data.co.uk/2015/2015.xlsx',
        'http://tennis-data.co.uk/2016/2016.xlsx', 'http://tennis-data.co.uk/2017/2017.xlsx', 'http://tennis-data.co.uk/2018/2018.xlsx', 'http://tennis-data.co.uk/2019/2019.xlsx',
        'http://tennis-data.co.uk/2020/2020.xlsx', 'http://tennis-data.co.uk/2021/2021.xlsx', 'http://tennis-data.co.uk/2022/2022.xlsx', 'http://tennis-data.co.uk/2023/2023.xlsx',
        'http://tennis-data.co.uk/2024/2024.xlsx', 'http://tennis-data.co.uk/2025/2025.xlsx']

# load each file and concatenate them into one dataframe
df = pd.DataFrame()
for i, elem in enumerate(links):
    if elem[-4:] == '.zip':
        content = requests.get(elem)
        zf = zipfile.ZipFile(BytesIO(content.content))
        temp = pd.read_excel(zf.open(zf.namelist()[0])) 
    else:
        temp = pd.read_excel(elem)
    df = pd.concat([df, temp], ignore_index=True)

# clean data
# some names have a trailing space
df['Winner'] = df['Winner'].str.strip()
df['Loser'] = df['Loser'].str.strip()

# some names do not have a full stop at the end - add it
df['Winner'] = [x + '.' if x[-1] != '.' else x for x in df['Winner']]
df['Loser'] = [x + '.' if x[-1] != '.' else x for x in df['Loser']]

# 15 matches have missing best of data - fill with 3
df['Best of'] = df['Best of'].fillna(3)

# only keep completed matches - removes retired, walkovers etc
df = df[df['Comment'] == 'Completed'].reset_index(drop=True)

# remove matches with incomplete data. Note that WPts and LPts are empty at the start, but we probably won't use them
# as the points system changes over time so rank is a more consistent metric
df = df[~df['WRank'].isnull()].reset_index(drop=True)
df = df[~df['LRank'].isnull()].reset_index(drop=True)
df = df.loc[df.LRank != 'NR']
df = df[~df['W1'].isnull()].reset_index(drop=True)
df = df[~df['W2'].isnull()].reset_index(drop=True)
df = df[~df['L1'].isnull()].reset_index(drop=True)
df = df[~df['L2'].isnull()].reset_index(drop=True)
df = df[~df['Wsets'].isnull()].reset_index(drop=True)
df = df[~df['Lsets'].isnull()].reset_index(drop=True)

# L3 and W3 sometimes have an empty string (in the same row as each other) - replace with NaN
df['W3'] = df['W3'].replace(' ', np.nan)
df['L3'] = df['L3'].replace(' ', np.nan)

# take only the interesting columns - removing only betting odds columns
df = df[['ATP','Location','Tournament','Date','Series','Court','Surface','Round','Best of','Winner','Loser','WRank','LRank','W1','L1','W2','L2','W3','L3','W4','L4','W5','L5','Wsets','Lsets']]

# save to single CSV file
df.to_csv('atp_tennis.csv')

c:\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\adamv\AppData\Local\Temp\ipykernel_18256\1455536942.py:51: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['W3'] = df['W3'].replace(' ', np.nan)
C:\Users\adamv\AppData\Local\